In [61]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import display
import warnings

import time

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# library options
%matplotlib inline
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

##### Loading Datasets

In [9]:
X_train = pd.read_parquet('../data/final/X_train.parquet')
X_test = pd.read_parquet('../data/final/X_test.parquet')
                      
y_test_reg = pd.read_parquet('../data/final/y_test.parquet')['sum_payments_package_key_ltv']
y_train_reg = pd.read_parquet('../data/final/y_train.parquet')['sum_payments_package_key_ltv']

##### Settings for K-Fold Cross-Validation

In [52]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)

# with transactional features
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train_reg, y_test_reg), axis=0)

# withhout transactional features
X_train_0 = X_train.drop(['sum_day_1','sum_day_2','sum_day_3','gradient','clumpiness'], axis=1).values
X_test_0 = X_test.drop(['sum_day_1','sum_day_2','sum_day_3','gradient','clumpiness'], axis=1).values

inputs_0 = np.concatenate((X_train_0, X_test_0), axis=0)



### AdaBoost Regression

##### K-Fold Cross-Validation

In [56]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    model = AdaBoostRegressor()
    model.fit(inputs[train], targets[train])

    y_predict = model.predict(inputs[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
    
scores_ada = scores

print('MSE ---- mean: ',scores_ada[0].mean(), '--- std: ',scores_ada[0].std())
print('MAE ---- mean: ',scores_ada[1].mean(), '--- std: ',scores_ada[1].std())
print('MSLE ---- mean: ',scores_ada[2].mean(), '--- std: ',scores_ada[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
MSE ---- mean:  970.383553503

##### Without transactional features

In [57]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs_0, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    model = AdaBoostRegressor()
    model.fit(inputs_0[train], targets[train])

    y_predict = model.predict(inputs_0[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
    
scores_ada = scores
print('--------------WITHOUT TRANSACTIONAL FEATURES-----------')
print('MSE ---- mean: ',scores_ada[0].mean(), '--- std: ',scores_ada[0].std())
print('MAE ---- mean: ',scores_ada[1].mean(), '--- std: ',scores_ada[1].std())
print('MSLE ---- mean: ',scores_ada[2].mean(), '--- std: ',scores_ada[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
--------------WITHOUT TRANSAC

### Gradient Boosting Regression

##### K-Fold Cross-Validation

In [58]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    model = GradientBoostingRegressor()
    model.fit(inputs[train], targets[train])

    y_predict = model.predict(inputs[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
    
scores_gb = scores

print('MSE ---- mean: ',scores_gb[0].mean(), '--- std: ',scores_gb[0].std())
print('MAE ---- mean: ',scores_gb[1].mean(), '--- std: ',scores_gb[1].std())
print('MSLE ---- mean: ',scores_gb[2].mean(), '--- std: ',scores_gb[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
MSE ---- mean:  731.946828130

##### Without transactional features

In [59]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs_0, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    model = GradientBoostingRegressor()
    model.fit(inputs_0[train], targets[train])

    y_predict = model.predict(inputs_0[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
    
scores_gb = scores
print('--------------WITHOUT TRANSACTIONAL FEATURES-----------')
print('MSE ---- mean: ',scores_gb[0].mean(), '--- std: ',scores_gb[0].std())
print('MAE ---- mean: ',scores_gb[1].mean(), '--- std: ',scores_gb[1].std())
print('MSLE ---- mean: ',scores_gb[2].mean(), '--- std: ',scores_gb[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
--------------WITHOUT TRANSAC

### XGBoost Regression

#### Hyperparameter Tuning with RandomSearch

##### I. Rough Tuning

In [70]:
params = { 'max_depth': [2,3,4,5,6,8],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': np.arange(0.5, 1.0, 0.1),
    'colsample_bytree': np.arange(0.4, 1.0, 0.1),
    'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
    'n_estimators': [50, 100, 150, 200],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0, 0.5, 1]}
    
xgbr = xgb.XGBRegressor()
    
xgb_search = RandomizedSearchCV(estimator=xgbr,
                    cv=4,
                    param_distributions=params,
                    scoring='neg_mean_squared_error',
                    n_iter=20,
                    verbose=2)
                    
xgb_search.fit(X_train.values, y_train_reg.values)
                    
print("Best parameters:", xgb_search.best_params_)
print("Lowest RMSE: ", (-xgb_search.best_score_))

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6, learning_rate=0.3, max_depth=6, n_estimators=150, reg_alpha=1, reg_lambda=0, subsample=0.7999999999999999; total time=  32.0s
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6, learning_rate=0.3, max_depth=6, n_estimators=150, reg_alpha=1, reg_lambda=0, subsample=0.7999999999999999; total time=  35.5s
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6, learning_rate=0.3, max_depth=6, n_estimators=150, reg_alpha=1, reg_lambda=0, subsample=0.7999999999999999; total time=  35.1s
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6, learning_rate=0.3, max_depth=6, n_estimators=150, reg_alpha=1, reg_lambda=0, subsample=0.7999999999999999; total time=  35.0s
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6, learning_rate=0.01, max_depth=4, n_estimators=200, reg_alpha=1, reg_lambda=0, subsample=0.6; total time=  34.2s
[CV] END colsample_bylevel=0.5, colsample_bytree=0.6,

In [79]:
df = pd.DataFrame(xgb_search.cv_results_)

df[['param_n_estimators',
'param_learning_rate',
'param_max_depth', 
'param_colsample_bytree', 
'param_colsample_bylevel',
'param_subsample',
'param_reg_alpha',
'param_reg_lambda',
'mean_test_score']].sort_values(by ='mean_test_score', ascending=False)[0:30]

,param_n_estimators,param_learning_rate,param_max_depth,param_colsample_bytree,param_colsample_bylevel,param_subsample,param_reg_alpha,param_reg_lambda,mean_test_score
14,150,0.1,3,0.9,0.8,0.6,1,0.5,-726.742105
9,150,0.2,2,0.6,0.9,0.6,1,0,-730.858411
5,50,0.3,3,0.5,0.9,0.7,0.5,0.5,-732.018837
10,150,0.1,3,0.4,0.4,0.6,0,1,-732.332168
19,100,0.1,3,0.5,0.5,0.5,0.5,0.5,-732.453772
8,50,0.1,4,0.6,0.8,0.6,0.5,0.5,-733.010468
6,50,0.2,2,0.5,0.4,0.6,1,1,-748.639804
13,150,0.3,5,0.5,0.7,0.8,0,0.5,-761.149100
17,100,0.2,8,0.8,0.6,0.8,0,0,-780.875134
18,100,0.2,8,0.4,0.5,0.5,1,1,-787.137057


##### II. Fine Tuning

In [80]:
params = { 'max_depth': [2,3],
    'learning_rate': np.arange(0.05,0.25,0.05),
    'subsample': np.arange(0.5, 0.7, 0.1),
    'colsample_bytree': np.arange(0.7, 1.1, 0.1),
    'colsample_bylevel': np.arange(0.6, 1.1, 0.1),
    'n_estimators': [125, 150, 175],
    'reg_alpha': np.arange(0, 1.4, 0.2),
    'reg_lambda': np.arange(0, 1.4, 0.2)}
    
xgbr = xgb.XGBRegressor()
    
xgb_search = RandomizedSearchCV(estimator=xgbr,
                    cv=4,
                    param_distributions=params,
                    scoring='neg_mean_squared_error',
                    n_iter=20,
                    verbose=2)
                    
xgb_search.fit(X_train.values, y_train_reg.values)
                    
print("Best parameters:", xgb_search.best_params_)
print("Lowest RMSE: ", (-xgb_search.best_score_))
df = pd.DataFrame(xgb_search.cv_results_)

df[['param_n_estimators',
'param_learning_rate',
'param_max_depth', 
'param_colsample_bytree', 
'param_colsample_bylevel',
'param_reg_alpha',
'param_reg_lambda',
'mean_test_score']].sort_values(by ='mean_test_score', ascending=False)[0:30]

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] END colsample_bylevel=0.7, colsample_bytree=0.9999999999999999, learning_rate=0.05, max_depth=2, n_estimators=125, reg_alpha=1.2000000000000002, reg_lambda=1.0, subsample=0.6; total time=  20.7s
[CV] END colsample_bylevel=0.7, colsample_bytree=0.9999999999999999, learning_rate=0.05, max_depth=2, n_estimators=125, reg_alpha=1.2000000000000002, reg_lambda=1.0, subsample=0.6; total time=  24.0s
[CV] END colsample_bylevel=0.7, colsample_bytree=0.9999999999999999, learning_rate=0.05, max_depth=2, n_estimators=125, reg_alpha=1.2000000000000002, reg_lambda=1.0, subsample=0.6; total time=  23.3s
[CV] END colsample_bylevel=0.7, colsample_bytree=0.9999999999999999, learning_rate=0.05, max_depth=2, n_estimators=125, reg_alpha=1.2000000000000002, reg_lambda=1.0, subsample=0.6; total time=  25.5s
[CV] END colsample_bylevel=0.9999999999999999, colsample_bytree=0.7999999999999999, learning_rate=0.05, max_depth=3, n_estimators=125, reg_

,param_n_estimators,param_learning_rate,param_max_depth,param_colsample_bytree,param_colsample_bylevel,param_reg_alpha,param_reg_lambda,mean_test_score
17,175,0.1,3,0.9,0.8,1.0,1.0,-725.531641
4,150,0.2,3,0.7,0.7,0.6,0.8,-728.298138
5,150,0.15,3,0.8,0.9,0.6,0.0,-728.846952
6,175,0.2,3,1.0,0.8,0.8,1.0,-728.849778
7,175,0.2,3,0.8,0.6,0.0,0.6,-728.875567
10,125,0.15,3,0.8,0.8,1.0,0.2,-729.106870
18,175,0.15,2,1.0,1.0,0.0,0.6,-730.014044
16,175,0.15,2,0.9,0.7,0.8,1.2,-730.363107
3,175,0.2,3,0.8,0.7,0.6,1.0,-730.788005
2,175,0.15,2,0.7,0.6,1.0,0.0,-731.898524


##### III. Fine-Fine Tuning

In [86]:
params = { 'max_depth': [3],
    'learning_rate': np.arange(0.1,0.25,0.05),
    'subsample': np.arange(0.5, 1, 0.1),
    'colsample_bytree': np.arange(0.7, 1.0, 0.1),
    'colsample_bylevel': np.arange(0.6, 1.0, 0.1),
    'n_estimators': [160,175,180],
    'reg_alpha': np.arange(0.6, 2, 0.2),
    'reg_lambda': np.arange(0.6, 2, 0.2)}
    
xgbr = xgb.XGBRegressor()
    
xgb_search = RandomizedSearchCV(estimator=xgbr,
                    cv=4,
                    param_distributions=params,
                    scoring='neg_mean_squared_error',
                    n_iter=20,
                    verbose=2)
                    
xgb_search.fit(X_train.values, y_train_reg.values)
                    
print("Best parameters:", xgb_search.best_params_)
print("Lowest RMSE: ", (-xgb_search.best_score_))
df = pd.DataFrame(xgb_search.cv_results_)

df[['param_n_estimators',
'param_learning_rate',
'param_max_depth', 
'param_colsample_bytree', 
'param_colsample_bylevel',
'param_subsample',
'param_reg_alpha',
'param_reg_lambda',
'mean_test_score']].sort_values(by ='mean_test_score', ascending=False)[0:30]

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=175, reg_alpha=1.2000000000000002, reg_lambda=1.8000000000000003, subsample=0.5; total time=  43.6s
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=175, reg_alpha=1.2000000000000002, reg_lambda=1.8000000000000003, subsample=0.5; total time= 1.1min
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=175, reg_alpha=1.2000000000000002, reg_lambda=1.8000000000000003, subsample=0.5; total time=  54.6s
[CV] END colsample_bylevel=0.8999999999999999, colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=175, reg_alpha=1.2000000000000002, reg_lambda=1.8000000000000003, subsample=0.5; total time=  48.7s
[CV] END colsample_bylevel=0.7, colsample_bytree=0.7999999999999999, learning_rate=

,param_n_estimators,param_learning_rate,param_max_depth,param_colsample_bytree,param_colsample_bylevel,param_subsample,param_reg_alpha,param_reg_lambda,mean_test_score
9,180,0.15,3,0.7,0.7,0.8,1.6,1.6,-724.234691
3,175,0.15,3,0.7,0.9,0.9,0.8,1.2,-724.415245
13,180,0.15,3,1.0,0.9,0.9,1.2,0.8,-724.553677
12,180,0.15,3,0.9,0.7,0.7,0.8,1.0,-724.615069
15,175,0.2,3,0.7,0.7,0.9,1.6,1.6,-724.733863
17,160,0.15,3,0.7,0.9,0.9,0.6,1.6,-724.795149
18,160,0.15,3,0.8,0.9,0.9,0.8,1.2,-725.101038
0,175,0.1,3,0.7,0.9,0.5,1.2,1.8,-725.772439
2,175,0.15,3,0.7,0.9,0.7,1.4,1.0,-726.085670
7,175,0.2,3,1.0,0.9,0.7,1.8,0.8,-726.183514


##### K-Fold Cross-Validation

In [87]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    param_best = xgb_search.best_params_
    model = xgb.XGBRegressor(**param_best)
    model.fit(inputs[train], targets[train])

    y_predict = model.predict(inputs[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
scores_xgb = scores

print('MSE ---- mean: ',scores_xgb[0].mean(), '--- std: ',scores_xgb[0].std())
print('MAE ---- mean: ',scores_xgb[1].mean(), '--- std: ',scores_xgb[1].std())
print('MSLE ---- mean: ',scores_xgb[2].mean(), '--- std: ',scores_xgb[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
MSE ---- mean:  727.003413360

##### Without transactional features

In [88]:
# K-fold Cross Validation model evaluation
fold_no = 1

# Initalize empty scores list
scores = np.zeros(shape=(3,num_folds))

for train, test in kfold.split(inputs_0, targets):
    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Chosing Model
    param_best = xgb_search.best_params_
    model = xgb.XGBRegressor(**param_best)
    model.fit(inputs_0[train], targets[train])

    y_predict = model.predict(inputs_0[test])
    y_predict_clipped = y_predict.clip(min=0) # setting min value to zero  (requirement for log_error)

    # filling scores list with the chosen sco
    scores[0,fold_no -1] = metrics.mean_squared_error(targets[test], y_predict)
    scores[1,fold_no -1] = metrics.mean_absolute_error(targets[test], y_predict)
    scores[2,fold_no -1] = metrics.mean_squared_log_error(targets[test], y_predict_clipped) # cannot handle negative input values as y_predict
    # Increase fold number
    fold_no = fold_no + 1
    
scores_xgb = scores
print('--------------WITHOUT TRANSACTIONAL FEATURES-----------')
print('MSE ---- mean: ',scores_xgb[0].mean(), '--- std: ',scores_xgb[0].std())
print('MAE ---- mean: ',scores_xgb[1].mean(), '--- std: ',scores_xgb[1].std())
print('MSLE ---- mean: ',scores_xgb[2].mean(), '--- std: ',scores_xgb[2].std())

------------------------------------------------------------------------
Training for fold 1 ...
------------------------------------------------------------------------
Training for fold 2 ...
------------------------------------------------------------------------
Training for fold 3 ...
------------------------------------------------------------------------
Training for fold 4 ...
------------------------------------------------------------------------
Training for fold 5 ...
------------------------------------------------------------------------
Training for fold 6 ...
------------------------------------------------------------------------
Training for fold 7 ...
------------------------------------------------------------------------
Training for fold 8 ...
------------------------------------------------------------------------
Training for fold 9 ...
------------------------------------------------------------------------
Training for fold 10 ...
--------------WITHOUT TRANSAC